In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import to_rgb
from elastica import *
from magneto_pyelastica import *
from magneto_pyelastica import utils
from typing import Dict, Sequence

SAVE_FIGURE = False
PLOT_FIGURE = False

class MagneticBeamSimulator(
    BaseSystemCollection, Constraints, Forcing, Damping, CallBacks
):
    pass


magnetic_beam_sim = MagneticBeamSimulator()

# setting up test params
n_elem = 50
start = np.zeros((3,))
direction = np.array([1.0, 0.0, 0.0])
normal = np.array([0.0, 1.0, 0.0])
base_length = 6.0
base_radius = 0.15
base_area = np.pi * base_radius**2
density = 5000
E = 1e6
I = np.pi / 4 * base_radius**4
poisson_ratio = 0.5
shear_modulus = E / (2 * poisson_ratio + 1.0)
base_radius = 0.15

# setting up magnetic properties
magnetization_density = 1e5
magnetic_field_angle = 2 * np.pi / 3
magnetic_field = 100
magnetization_direction = np.ones((n_elem)) * direction.reshape(3, 1)

magnetic_rod = CosseratRod.straight_rod(
    n_elem,
    start,
    direction,
    normal,
    base_length,
    base_radius,
    density,
    youngs_modulus=E,
    shear_modulus=shear_modulus,
)
magnetic_beam_sim.append(magnetic_rod)

# Add boundary conditions, one end of rod is clamped
magnetic_beam_sim.constrain(magnetic_rod).using(
    OneEndFixedBC, constrained_position_idx=(0,), constrained_director_idx=(0,)
)

# Set the constant magnetic field object
magnetic_field_amplitude = magnetic_field * np.array(
    [np.cos(magnetic_field_angle), np.sin(magnetic_field_angle), 0]
)
magnetic_field_object = ConstantMagneticField(
    magnetic_field_amplitude, ramp_interval=500.0, start_time=0.0, end_time=1000.0
)

# Apply magnetic forces
magnetic_beam_sim.add_forcing_to(magnetic_rod).using(
    MagneticForces,
    external_magnetic_field=magnetic_field_object,
    magnetization_density=magnetization_density,
    magnetization_direction=magnetization_direction,
    rod_volume=magnetic_rod.volume,
    rod_director_collection=magnetic_rod.director_collection,
)

# Add callbacks
class MagneticBeamCallBack(CallBackBaseClass):
    def __init__(self, step_skip: int, callback_params: dict):
        CallBackBaseClass.__init__(self)
        self.every = step_skip
        self.callback_params = callback_params

    def make_callback(self, system, time, current_step: int):
        if current_step % self.every == 0:
            self.callback_params["time"].append(time)
            self.callback_params["step"].append(current_step)
            self.callback_params["position"].append(system.position_collection.copy())
            self.callback_params["velocity_norm"].append(
                np.linalg.norm(system.velocity_collection)
            )
        return


# add damping
dl = base_length / n_elem
dt = 0.05 * dl
damping_constant = 1.0
magnetic_beam_sim.dampen(magnetic_rod).using(
    AnalyticalLinearDamper,
    damping_constant=damping_constant,
    time_step=dt,
)

# Add call back for plotting time history of the rod
post_processing_dict = defaultdict(list)
magnetic_beam_sim.collect_diagnostics(magnetic_rod).using(
    MagneticBeamCallBack, step_skip=100, callback_params=post_processing_dict
)


In [2]:

magnetic_beam_sim.finalize()
timestepper = PositionVerlet()
final_time = 1000.0
total_steps = int(final_time / dt)
print("Total steps", total_steps)
integrate(timestepper, magnetic_beam_sim, final_time, total_steps)

if PLOT_FIGURE:
    with plt.style.context("ggplot"):
        fig = plt.figure(figsize=(10, 8), frameon=True, dpi=150)
        ax = fig.add_subplot(111)
        ax.plot(
            magnetic_rod.position_collection[0, ...],
            magnetic_rod.position_collection[1, ...],
            lw=2,
            c=to_rgb("xkcd:bluish"),
        )
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        fig.show()

        fig = plt.figure(figsize=(10, 8), frameon=True, dpi=150)
        ax = fig.add_subplot(111)
        ax.semilogy(
            np.asarray(post_processing_dict["time"]),
            np.asarray(post_processing_dict["velocity_norm"]),
            lw=2,
            c=to_rgb("xkcd:bluish"),
        )
        ax.set_xlabel("t")
        ax.set_ylabel("|v|")
        fig.show()

        plt.show()  # block
    if SAVE_FIGURE:
        fig.savefig("Magnetic_beam_profile: N=" + str(magnetic_rod.n_elems) + ".png")

Total steps 166666


  0%|          | 0/166666 [00:00<?, ?it/s]

100%|██████████| 166666/166666 [00:07<00:00, 21201.19it/s]

Final time of simulation is :  999.9999999957777


In [3]:
from IPython.display import Video
from tqdm import tqdm


def plot_video_2D(plot_params: dict, video_name="video.mp4", margin=0.2, fps=15):
    from matplotlib import pyplot as plt
    import matplotlib.animation as manimation

    t = np.array(plot_params["time"])
    positions_over_time = np.array(plot_params["position"])
    total_time = int(np.around(t[..., -1], 1))
    total_frames = fps * total_time
    step = round(len(t) / total_frames)

    print("creating video -- this can take a few minutes")
    FFMpegWriter = manimation.writers["ffmpeg"]
    metadata = dict(title="Movie Test", artist="Matplotlib", comment="Movie support!")
    writer = FFMpegWriter(fps=fps, metadata=metadata)

    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.axis("equal")
    rod_lines_2d = ax.plot(
        positions_over_time[0][2], positions_over_time[0][0], linewidth=3
    )[0]
    ax.set_xlim([0 - margin, 3 + margin])
    ax.set_ylim([-1.5 - margin, 1.5 + margin])
    with writer.saving(fig, video_name, dpi=100):
        with plt.style.context("seaborn-whitegrid"):
            print(len(t), " len(t)")
            print(total_time, " total_time")
            print(total_frames, " total_frames")
            print("step size: ", step, " len(t) / total_frames")
            for time in range(1, len(t), 1):
                rod_lines_2d.set_xdata(positions_over_time[time][2])
                rod_lines_2d.set_ydata(positions_over_time[time][0])

                writer.grab_frame()
    plt.close(fig)

filename = "magnetic_beam.mp4"
plot_video_2D(post_processing_dict, filename, margin=0.2, fps=15)
Video(filename)

creating video -- this can take a few minutes
1667  len(t)
999  total_time
14985  total_frames
step size:  0  len(t) / total_frames
